READ FILE

In [1]:
import os 

current_directory = os.getcwd()
file_name = "PeakUp.txt"
file_path = os.path.join(current_directory, file_name)

with open(file_path, 'r', encoding = "utf-8", errors = "replace") as file:
    content = file.read()
print(content)

my_api_key = os.get.environ('my_api_key')
my_azure_key = os.get.environ('my_azure_key')
my_and_point = os.get.environ('my_and_point')

İzinler ve Kullanım Alanları
PEAKUP’ta izinler onaya tabi değil! İhtiyacın olduğunda Leave Management (https://leave.peakup.org/) 
üzerinden ilgili izin tipinde giriş yapabilir, planlamaların sağlıklı yürütülebilmesi için ekip arkadaşlarına 
ve birim yöneticine haber vermenin ardından onay beklemeksizin izin hakkını kullanabilirsin. 
Peki hangi izin hakkını ne amaç için kullanmalıyız? 
Yıllık Ücretli İzin: Çalışma süresi 1 yıldan 5 yıla kadar (5 yıl dahil) olan çalışma arkadaşlarımıza 14 iş günü 
ve 5 yıldan fazla olanlara 20 iş günü yıllık ücretli izin verilir. Bu izin hakkını tatil ve dinlenmek için 
kullanabilirsin. Yıl içinde kullanmadığın izin günleri bir sonraki yıla devredilir. 
Evlenme İzni: Evlenen çalışma arkadaşlarımıza 5 (beş) iş günü evlenme izni verilir.
Mazeret İzni: İş Kanunu kapsamında yıllık iznine ek olarak 6 (altı) gün mazeret izni hakkı sunuyoruz. Bu 
iznini saatlik ve günlük olarak kullanabilirsin. Bankaya gitmek, doktor randevusu, pasaport randevusu, 
uyuyakalmak

CHUNCKING

In [2]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    separators= [" ", ",", "\n"],
    length_function = len

)
texts = text_splitter.create_documents([content])

for i in range(5):

    print(f"Text: {i+1}")
    print(texts[i].page_content)

Text: 1
İzinler ve Kullanım Alanları
PEAKUP’ta izinler onaya tabi değil! İhtiyacın olduğunda Leave Management (https://leave.peakup.org/) 
üzerinden ilgili izin tipinde giriş yapabilir, planlamaların sağlıklı yürütülebilmesi için ekip arkadaşlarına 
ve birim yöneticine haber vermenin ardından onay beklemeksizin izin hakkını kullanabilirsin. 
Peki hangi izin hakkını ne amaç için kullanmalıyız? 
Yıllık Ücretli İzin: Çalışma süresi 1 yıldan 5 yıla kadar (5 yıl dahil) olan çalışma arkadaşlarımıza 14 iş günü 
ve 5 yıldan fazla olanlara 20 iş günü yıllık ücretli izin verilir. Bu izin hakkını tatil ve dinlenmek için 
kullanabilirsin. Yıl içinde kullanmadığın izin günleri bir sonraki yıla devredilir. 
Evlenme İzni: Evlenen çalışma arkadaşlarımıza 5 (beş) iş günü evlenme izni verilir.
Mazeret İzni: İş Kanunu kapsamında yıllık iznine ek olarak 6 (altı) gün mazeret izni hakkı sunuyoruz. Bu 
iznini saatlik ve günlük olarak kullanabilirsin. Bankaya gitmek, doktor randevusu, pasaport randevusu,
Text

EMBEDDING

In [3]:
import uuid
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader

client = OpenAI(api_key = my_api_key)

def generate_embeddings(text):
    response = client.embeddings.create(
        input = [text],
        model = "text-embedding-3-small")
    return response.data[0].embedding

embedding_collection = []

for chunk in texts:
    embedding_collection.append({

        'id':str(uuid.uuid4()),
        'content':chunk.page_content,
        'contentVector' : generate_embeddings(chunk.page_content)
    })

print(embedding_collection[0])

{'id': 'ff00457d-ba29-4fe9-a1a6-e5916110131d', 'content': 'İzinler ve Kullanım Alanları\nPEAKUP’ta izinler onaya tabi değil! İhtiyacın olduğunda Leave Management (https://leave.peakup.org/) \nüzerinden ilgili izin tipinde giriş yapabilir, planlamaların sağlıklı yürütülebilmesi için ekip arkadaşlarına \nve birim yöneticine haber vermenin ardından onay beklemeksizin izin hakkını kullanabilirsin. \nPeki hangi izin hakkını ne amaç için kullanmalıyız? \nYıllık Ücretli İzin: Çalışma süresi 1 yıldan 5 yıla kadar (5 yıl dahil) olan çalışma arkadaşlarımıza 14 iş günü \nve 5 yıldan fazla olanlara 20 iş günü yıllık ücretli izin verilir. Bu izin hakkını tatil ve dinlenmek için \nkullanabilirsin. Yıl içinde kullanmadığın izin günleri bir sonraki yıla devredilir. \nEvlenme İzni: Evlenen çalışma arkadaşlarımıza 5 (beş) iş günü evlenme izni verilir.\nMazeret İzni: İş Kanunu kapsamında yıllık iznine ek olarak 6 (altı) gün mazeret izni hakkı sunuyoruz. Bu \niznini saatlik ve günlük olarak kullanabilirsi

CREATING INDEX

In [13]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,
    ComplexField,
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,
    VectorSearchAlgorithmConfiguration,
    VectorSearch,
    SemanticSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SemanticPrioritizedFields,
)
credential = AzureKeyCredential(my_azure_key) 
index_client = SearchIndexClient(endpoint=my_and_point, credential=credential)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retriveable=True),
    SearchField(name="contentVector",type= SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
]

vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ]
)
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="content")]
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])

index = SearchIndex(name="peakupdemoindex", fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)

UPLOAD EMBEDDİNGS

In [14]:
import uuid
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint=my_and_point, index_name="peakupdemoindex", credential=credential)
result = search_client.upload_documents(embedding_collection)


PERFORM VECTOR SIMILARITY

In [9]:
from azure.search.documents.models import VectorizedQuery
from itertools import tee

query = "Yıllık ücretli izin ne kadar?"
embedding = client.embeddings.create(input=query, model='text-embedding-3-small').data[0].embedding
search_client = SearchClient(my_and_point, "peakupdemoindex", AzureKeyCredential(my_azure_key))

results = search_client.search(
    search_text="",
    vector_queries= [VectorizedQuery(vector=embedding,k_nearest_neighbors=3,fields="contentVector")],
    select=["id", "content"],
)

result, results_backup = tee(results)

for result in results_backup:
    print(f"Score:{result["@search.score"]}")
    print(f"Answer:{result['content']}")


Score:0.74209136
Answer:İzinler ve Kullanım Alanları
PEAKUP’ta izinler onaya tabi değil! İhtiyacın olduğunda Leave Management (https://leave.peakup.org/) 
üzerinden ilgili izin tipinde giriş yapabilir, planlamaların sağlıklı yürütülebilmesi için ekip arkadaşlarına 
ve birim yöneticine haber vermenin ardından onay beklemeksizin izin hakkını kullanabilirsin. 
Peki hangi izin hakkını ne amaç için kullanmalıyız? 
Yıllık Ücretli İzin: Çalışma süresi 1 yıldan 5 yıla kadar (5 yıl dahil) olan çalışma arkadaşlarımıza 14 iş günü 
ve 5 yıldan fazla olanlara 20 iş günü yıllık ücretli izin verilir. Bu izin hakkını tatil ve dinlenmek için 
kullanabilirsin. Yıl içinde kullanmadığın izin günleri bir sonraki yıla devredilir. 
Evlenme İzni: Evlenen çalışma arkadaşlarımıza 5 (beş) iş günü evlenme izni verilir.
Mazeret İzni: İş Kanunu kapsamında yıllık iznine ek olarak 6 (altı) gün mazeret izni hakkı sunuyoruz. Bu 
iznini saatlik ve günlük olarak kullanabilirsin. Bankaya gitmek, doktor randevusu, pasaport

PERFORM SEMANTIC HYBRID SEARCH 

In [12]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType
from itertools import tee

query = "Yıllık ücretli izin ne kadar?"
embedding = client.embeddings.create(input=query, model='text-embedding-3-small').data[0].embedding
search_client = SearchClient(my_and_point, "peakupdemoindex", AzureKeyCredential(my_azure_key))

results = search_client.search(
    search_text="",
    vector_queries= [VectorizedQuery(vector=embedding,k_nearest_neighbors=3,fields="contentVector")],
    select=["id", "content"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

result, results_backup = tee(results)
print(f"Query: {query}")
for result in results_backup:
    print(f"Score: {result["@search.score"]}")
    print(result['content'])



Query: Yıllık ücretli izin ne kadar?
Score: 0.74209136
İzinler ve Kullanım Alanları
PEAKUP’ta izinler onaya tabi değil! İhtiyacın olduğunda Leave Management (https://leave.peakup.org/) 
üzerinden ilgili izin tipinde giriş yapabilir, planlamaların sağlıklı yürütülebilmesi için ekip arkadaşlarına 
ve birim yöneticine haber vermenin ardından onay beklemeksizin izin hakkını kullanabilirsin. 
Peki hangi izin hakkını ne amaç için kullanmalıyız? 
Yıllık Ücretli İzin: Çalışma süresi 1 yıldan 5 yıla kadar (5 yıl dahil) olan çalışma arkadaşlarımıza 14 iş günü 
ve 5 yıldan fazla olanlara 20 iş günü yıllık ücretli izin verilir. Bu izin hakkını tatil ve dinlenmek için 
kullanabilirsin. Yıl içinde kullanmadığın izin günleri bir sonraki yıla devredilir. 
Evlenme İzni: Evlenen çalışma arkadaşlarımıza 5 (beş) iş günü evlenme izni verilir.
Mazeret İzni: İş Kanunu kapsamında yıllık iznine ek olarak 6 (altı) gün mazeret izni hakkı sunuyoruz. Bu 
iznini saatlik ve günlük olarak kullanabilirsin. Bankaya git

RAG

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature = 0, openai_api_key = my_api_key)

query = "Zorunlu stak kaç gün?"

rag_template = ''' 
###
CONTEXT:
"""
{context}
"""
###
RULES:
You ara a helpful agent that answers questions.
Learn from the CONTEXT and asnwer the following questions.

###
{question}

'''

rag_prompt = ChatPromptTemplate.from_messages(

    messages= ["context", "question"]
)

llm(rag_prompt.format_messages(question = query, context=next(results)['content'] ))
